### Strategy Design Pattern
Strategy is a behavioral design pattern that lets you define a family of algorithms, put each of them into a separate class, and make their objects interchangeable.

In [ ]:
# Using the Composition Pattern (has-a) relationship
from typing import Protocol, Iterable

class SortStrategy(Protocol):
    def run_algorithm(self, iterable: Iterable) -> Iterable: ...


class ASCStrategy:
    def run_algorithm(self, iterable: Iterable) -> Iterable:
        return sorted(iterable)


class DSCStrategy:
    def run_algorithm(self, iterable: Iterable) -> Iterable:
        return sorted(iterable, reverse=True)


class Program:
    def __init__(self, strategy: SortStrategy, iterable: Iterable):
        self._strategy = strategy
        self.iterable = iterable

    @property
    def strategy(self) -> SortStrategy:
        return self._strategy
    
    @strategy.setter
    def strategy(self, strategy: SortStrategy) -> None:
        if self._strategy == strategy:
            return
        print(
            (
                f"Strategy: {self._strategy.__class__.__name__} "
                f"changed to {strategy.__class__.__name__}"
            )
        )
        self._strategy = strategy

    def exec(self) -> Iterable:
        return self._strategy.run_algorithm(self.iterable)

p = Program(ASCStrategy(), [1, 3, 2, 4, 5])
print(p.exec())

p.strategy = DSCStrategy()
print(p.exec())

[1, 2, 3, 4, 5]
Strategy: ASCStrategy changed to DSCStrategy
[5, 4, 3, 2, 1]


In [ ]:
# Using Inheritance (is-a) relationship
from abc import ABC

class StStrategy(ABC):
    def run_algorithm(self, iterable: Iterable) -> Iterable:
        raise NotImplementedError("You should implement this method!")

class StASCStrategy(StStrategy):
    def run_algorithm(self, iterable: Iterable) -> Iterable:
        return sorted(iterable)

class StDSCStrategy(StStrategy):
    def run_algorithm(self, iterable: Iterable) -> Iterable:
        return sorted(iterable, reverse=True)

class StProgram:
    def __init__(self, strategy: StStrategy, iterable: Iterable):
        self._strategy = strategy
        self.iterable = iterable

    @property
    def strategy(self) -> StStrategy:
        return self._strategy
    
    @strategy.setter
    def strategy(self, strategy: StStrategy) -> None:
        if self._strategy == strategy:
            return
        print(
            (
                f"Strategy: {self._strategy.__class__.__name__} "
                f"changed to {strategy.__class__.__name__}"
            )
        )
        self._strategy = strategy

    def exec(self) -> Iterable:
        return self._strategy.run_algorithm(self.iterable)

p = StProgram(StASCStrategy(), [1, 3, 2, 4, 5])
print(p.exec())
p.strategy = StDSCStrategy()
print(p.exec())

[1, 2, 3, 4, 5]
Strategy: StASCStrategy changed to StDSCStrategy
[5, 4, 3, 2, 1]


### State Pattern
**State** is a behavioral design pattern that lets an object alter its behavior when its internal state changes. It appears as if the object changed its class. Just like a `finite state machine`.

In [8]:
from abc import ABC


class Context:
    pass


class State(ABC):

    @property
    def context(self):
        return self._context

    @context.setter
    def context(self, context: Context):
        self._context = context

    def exec(self):
        raise NotImplementedError("You need to implement this method")

class Init(State):
    def exec(self):
        print("Starting a new task...")
        self.context.state = InProgress()

class InProgress(State):
    pass

In [11]:
x = [x.strip() for x in "This, is, a, task".strip().split(",")]
x

['This', 'is', 'a', 'task']

In [17]:
from datetime import datetime, timezone
datetime.now(timezone.utc).year

2025

In [18]:
"2025".isdigit()

True

In [1]:
%pip install redis

  Using cached redis-5.2.1-py3-none-any.whl.metadata (9.1 kB)
Using cached redis-5.2.1-py3-none-any.whl (261 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import redis
r = redis.Redis(decode_responses=True, max_connections=20)
r.ping()

True

In [35]:
r.set("hello", "world")

True

In [24]:
r.acl_cat()

['keyspace',
 'read',
 'write',
 'set',
 'sortedset',
 'list',
 'hash',
 'string',
 'bitmap',
 'hyperloglog',
 'geo',
 'stream',
 'pubsub',
 'admin',
 'fast',
 'slow',
 'blocking',
 'dangerous',
 'connection',
 'transaction',
 'scripting']

In [4]:
r.acl_cat(category="list")

['sort_ro',
 'blpop',
 'rpush',
 'blmpop',
 'lmove',
 'lrem',
 'llen',
 'lpop',
 'lmpop',
 'blmove',
 'linsert',
 'lset',
 'rpushx',
 'sort',
 'lpush',
 'brpop',
 'lpos',
 'brpoplpush',
 'rpop',
 'ltrim',
 'lpushx',
 'lrange',
 'lindex',
 'rpoplpush']

In [33]:
r.acl_list()

['user default on nopass sanitize-payload ~* &* +@all']

In [7]:
PLANETS = [
    "Mercury", "Mercury", "Venus", "Earth", "Earth", "Mars",
    "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto"
]
PLANET_LIST_KEY = "ru102py:planets:list"
PLANET_SET_KEY = "ru102py:planets:set"
EARTH_KEY = "earth"

In [8]:
def test_redis_list(redis: redis.Redis):
    assert len(PLANETS ) == 11

    # Add all test planets to Redis list
    result = redis.rpush(PLANET_LIST_KEY, *PLANETS)

    # Check that the length of the list in Redis is the same
    assert result == len(PLANETS)

    # Get the planets from the list
    # NOTEL LRANGE is an O(n) command. Be careful running it with high-cardinlity sets
    planets = redis.lrange(PLANET_LIST_KEY, 0, -1)
    assert planets == PLANETS

    # Remove the elements that we know are duplicates
    redis.lrem(PLANET_LIST_KEY, 1, "Mercury")
    redis.lrem(PLANET_LIST_KEY, 1, "Earth")

    planet = redis.rpop(PLANET_LIST_KEY)
    assert planet == "Pluto"

    assert redis.llen(PLANET_LIST_KEY) == 8

test_redis_list(r)

In [12]:
def test_redis_set(redis: redis.Redis):
    # Add all test planets to Redis set
    redis.sadd(PLANET_SET_KEY, *PLANETS)

    # Return the cardinality of the set
    assert redis.scard(PLANET_SET_KEY) == 9

    # Get the planets from the set
    # NOTE: SMEMBERS is an O(n) command. Be careful running it with high-cardinlity sets
    # and large sets
    # Planets are not returned in any particular order
    # and duplicates are removed
    assert redis.smembers(PLANET_SET_KEY) == set(PLANETS)

    # Remove the elements that we know are duplicates
    res = redis.srem(PLANET_SET_KEY, "Pluto")
    assert res == 1

    print(redis.sscan(PLANET_SET_KEY))

test_redis_set(r)

(0, ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune'])


In [19]:
data_2 = { "store": {
    "book": [ 
      { "category": "reference",
        "author": "Nigel Rees",
        "title": "Sayings of the Century",
        "price": 8.95
      },
      { "category": "fiction",
        "author": "Evelyn Waugh",
        "title": "Sword of Honour",
        "price": 12.99
      },
      { "category": "fiction",
        "author": "Herman Melville",
        "title": "Moby Dick",
        "isbn": "0-553-21311-3",
        "price": 8.99
      },
      { "category": "fiction",
        "author": "J. R. R. Tolkien",
        "title": "The Lord of the Rings",
        "isbn": "0-395-19395-8",
        "price": 22.99
      }
    ],
    "bicycle": {
      "color": "red",
      "price": 19.95
    }
  }
}

In [22]:
import json

rj = redis.StrictRedis(port=6399)
data = {
    'foo': 'bar',
    'ans': 42
}
rj.execute_command("JSON.SET", "object_", ".", json.dumps(data))
reply = rj.execute_command("JSON.GET", "object")

rj.execute_command("JSON.SET", "object_2", ".", json.dumps(data_2))
res = rj.execute_command("JSON.GET", "..author")
print(res)


None


In [ ]:
# rj.execute_command("PFADD", "bikes")
rj.execute_command("PFADD", "bikes", "Hyperion","Deimos","Phoebe", "Quaoar")

0

In [27]:
rj.execute_command("PFADD", "bikes", "Deimos")

0

In [28]:
rj.execute_command("PFCOUNT", "bikes")

4

In [29]:
rj.execute_command("PFADD", "commuter_bikes", "Salacia", "Mimas", "Quaoar")

1

In [30]:
rj.execute_command("PFCOUNT", "commuter_bikes", "bikes")

6

In [31]:
rj.execute_command("PFMERGE", "all_bikes", "bikes", "commuter_bikes")

True

In [32]:
rj.execute_command("PFCOUNT", "all_bikes")

6

In [33]:
dir(rj)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__del__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__parameters__',
 '__protocol_attrs__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_disconnect_raise',
 '_eval',
 '_evalsha',
 '_execute_command',
 '_fcall',
 '_georadiusgeneric',
 '_geosearchgeneric',
 '_is_protocol',
 '_is_runtime_protocol',
 '_send_command_parse_response',
 '_tfcall',
 '_zaggregate',
 '_zrange',
 'acl_cat',
 'acl_deluser',
 'acl_dryrun',
 'acl_genpass',
 'acl_getuser',
 'acl_help',
 'acl_list',
 'acl_load',
 'acl_log',
 'acl_log_r